# Multi-Factor Model: Short-Term Reversal

In this notebook, you will create a short-term reversal factor. And then combine it with the momentum factor to generate a multi-factor model

The following steps are performed while creating the multi-factor model.
1. Import the libraries and data
2. Calculate the daily percentage change
3. Create a short-term reversal based factor
4. Create a multi factor portfolio

### Import the libraries and data

#### Import the libraries
You can first start by importing the libraries necessary which will be used later to get the data, store it, and create the factor based portfolio

In [1]:
# Importing modules
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import datetime as dt 

# Need to import the matplot_converters from pandas
# whenever you try to plot the columns of a dataframe using a for loop
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#### Download the data

In [2]:
df = pd.read_csv('data/Stock_Prices_2012_To_2017_Factor.csv', index_col=[0])

# Convert the index to datetime format
df.index = pd.to_datetime(df.index, format="%Y-%m-%d")#.date

# Load the momentum factor created in the previous notebook
momentum_performance = pd.read_csv('data/Momentum_Performance_2012_To_2017.csv', index_col=[0])

# Convert the index to datatime format
momentum_performance.index = pd.to_datetime(momentum_performance.index, format='%d-%m-%Y')